In [1]:
import numpy as np
from termcolor import colored

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RepeatVector , SimpleRNN , TimeDistributed, Dense
from tensorflow.keras.callbacks import LambdaCallback , EarlyStopping

In [2]:
all_chars = '0123456789+-*'
num_features = len(all_chars)
print(num_features)
max_length = 5

13


In [3]:
char_to_index = dict((c,i) for i,c in enumerate(all_chars))
index_to_char = dict((i,c) for i,c in enumerate(all_chars))
print(char_to_index)
print(index_to_char)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '+': 10, '-': 11, '*': 12}
{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '+', 11: '-', 12: '*'}


In [4]:
def generate_data():
  first_num = np.random.randint(0,100)
  second_num = np.random.randint(0,100)
  sign = np.random.randint(0,4)
  if sign == 0:
    example = str(first_num) + '+' + str(second_num)
    label = str(first_num + second_num)
  elif sign == 1:
    example = str(first_num) + '-' + str(second_num)
    label = str(first_num - second_num)
  elif sign == 2:
    example = str(first_num) + '*' + str(second_num)
    label = str(first_num * second_num)
  elif sign == 3:
    example = str(first_num) + '*' + str(second_num)
    label = str(first_num * second_num)

  return example,label

In [5]:
e,l = generate_data()
print(e,l)

29-19 10


In [6]:
def generate_dataset(num_examples=5000):
  x_train = np.zeros((num_examples,max_length,num_features))
  y_train = np.zeros((num_examples,max_length,num_features))

  for i in range(num_examples):
    e,l = generate_data()
    e,l = vectorized(e,l)
    x_train[i] = e
    y_train[i] = l

  return(x_train,y_train)

In [7]:
def vectorized(example,label):
  x = np.zeros((max_length,num_features))
  y = np.zeros((max_length,num_features))
  diff_x = max_length - len(example)
  diff_y = max_length - len(label)

  for i,c in enumerate(example):
    x[diff_x+i,char_to_index[c]] = 1

  for i in range(diff_x):
    x[i,char_to_index['0']] = 1

  for i,c in enumerate(label):
    y[diff_y+i,char_to_index[c]] = 1

  for i in range(diff_y):
    y[i,char_to_index['0']] = 1

  return x,y

In [8]:
a,b = vectorized('5+5','10')

In [9]:
def devectorized(label):
  result = [index_to_char[np.argmax(vec)] for i,vec in enumerate(label)]
  return ''.join(result)

In [10]:
print(devectorized(b))
print(devectorized(a))

00010
005+5


In [11]:
def strip_zeros(example):
  out = ''
  encountered_non_zero = False
  non_zero = '123456789-'
  for i in example:
    if i in non_zero :
      encountered_non_zero = True
    if encountered_non_zero == True:
      out += i
  return out

In [12]:
strip_zeros('0010')

'10'

In [13]:
hidden_units = 128

In [14]:
x_train , y_train = generate_dataset(100000)

In [15]:
model = Sequential([
    SimpleRNN(hidden_units,input_shape=(None,num_features)),
    RepeatVector(max_length),
    SimpleRNN(hidden_units,return_sequences= True),
    TimeDistributed(Dense(num_features,activation='softmax'))
])

model.compile(
    loss ='categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
              )
model.summary()

D:\Github\PlayingWithRNNs\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 128)                 │          18,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector (RepeatVector)         │ (None, 5, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 5, 128)              │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 5, 13)               │           1,677 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 52,749 (206.05 KB)

 Trainable params: 52,749 (206.05 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
early_stopping = EarlyStopping(monitor = 'val_loss',patience=20)
simple_logger = LambdaCallback(
    on_epoch_end=lambda e, l: print('{:.2f}'.format(l['val_accuracy']), end=' _ ')
)


model.fit(x_train,y_train,epochs=1000,validation_split=0.2,verbose=False,batch_size=512,callbacks=[early_stopping,simple_logger])

0.57 _ 0.62 _ 0.66 _ 0.68 _ 0.70 _ 0.72 _ 0.76 _ 0.77 _ 0.78 _ 0.81 _ 0.82 _ 0.83 _ 0.84 _ 0.85 _ 0.86 _ 0.86 _ 0.86 _ 0.87 _ 0.87 _ 0.88 _ 0.88 _ 0.88 _ 0.89 _ 0.89 _ 0.89 _ 0.89 _ 0.89 _ 0.90 _ 0.89 _ 0.90 _ 0.90 _ 0.91 _ 0.91 _ 0.91 _ 0.91 _ 0.92 _ 0.91 _ 0.92 _ 0.92 _ 0.90 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.93 _ 0.92 _ 0.93 _ 0.92 _ 0.93 _ 0.93 _ 0.93 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.93 _ 0.94 _ 0.95 _ 0.94 _ 0.95 _ 0.94 _ 0.95 _ 0.94 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.94 _ 0.95 _ 0.95 _ 0.95 _ 0.96 _ 0.95 _ 0.95 _ 0.95 _ 0.96 _ 0.95 _ 0.95 _ 0.96 _ 0.96 _ 0.96 _ 0.94 _ 0.95 _ 0.96 _ 0.96 _ 0.96 _ 0.96 _ 0.96 _ 0.96 _ 0.96 _ 0.96 _ 0.97 _ 0.96 _ 0.97 _ 0.96 _ 0.96 _ 0.96 _ 0.97 _ 0.93 _ 0.97 _ 0.97 _ 0.97 _ 0.97 _ 0.96 _ 0.97 _ 0.97 _ 0.97 _ 0.96 _ 0.96 _ 0.97 _ 0.98 _ 0.97 _ 0.97 _ 0.97 _ 0.97 _ 0.97 _ 0.96 _ 0.97 _ 0.97 _ 0.97 _ 0.98 _ 0.98 _ 0.98 _ 0.97 _ 0.97 _ 0.97 _ 0.98 _ 0.98 _ 0.98 _ 0.97 _ 0.97 _ 0.98 _ 0.97 _ 0.98 _ 0.97 _ 0.97 _ 0.98 _ 0.98 _ 0.97 _ 0.97 _

In [17]:
x_test, y_test = generate_dataset(num_examples=20)
preds = model.predict(x_test)
full_seq_acc = 0

for i, pred in enumerate(preds):
    pred_str = strip_zeros(devectorized(pred))
    y_test_str = strip_zeros(devectorized(y_test[i]))
    x_test_str = strip_zeros(devectorized(x_test[i]))
    col = 'green' if pred_str == y_test_str else 'red'
    full_seq_acc += 1/len(preds) * int(pred_str == y_test_str)
    outstring = 'Input: {}, Out: {}, Pred: {}'.format(x_test_str, y_test_str, pred_str)
    print(colored(outstring, col))
print('\nFull sequence accuracy: {:.3f} %'.format(100 * full_seq_acc))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Input: 20-49, Out: -29, Pred: -29
Input: 56*92, Out: 5152, Pred: 5152
Input: 41+14, Out: 55, Pred: 55
Input: 56*25, Out: 1400, Pred: 1400
Input: 73+34, Out: 107, Pred: 107
Input: 22+90, Out: 112, Pred: 112
Input: 38*69, Out: 2622, Pred: 2622
Input: 81-65, Out: 16, Pred: 16
Input: 91+18, Out: 109, Pred: 109
Input: 92-32, Out: 60, Pred: 60
Input: 44+59, Out: 103, Pred: 103
Input: 29*77, Out: 2233, Pred: 2233
Input: 23*20, Out: 460, Pred: 460
Input: 68-90, Out: -22, Pred: -22
Input: 6+43, Out: 49, Pred: 49
Input: 17-24, Out: -7, Pred: -7
Input: 50-7, Out: 43, Pred: 43
Input: 88*12, Out: 1056, Pred: 1056
Input: 82*52, Out: 4264, Pred: 4264
Input: 18*38, Out: 684, Pred: 684

Full sequence accuracy: 100.000 %


In [19]:
#Saving the trained model

model.save('../saved_model/rnn_arithmetic_model.h5')
